In [ ]:
import pandas as pd
import requests
import os
from datetime import datetime
from dotenv import load_dotenv
import sys
# hack to get around notebook being not in main directory
sys.path.append("g:/My Drive/DS Projects/Morning_Brief")
from src.utils.config import API_KEY_OPEN_WEATHER


In [ ]:
from src.weather import *

print(get_forecast_by_city("London", 10))

print(get_forecast_by_zip_code("e1","gb", 10))

[{'time': '2025-04-23 12:00:00', 'temp': 9.69, 'wind_speed': 4.21, 'humidity': 87, 'weather_desc': 'light rain', 'cloudiness': 75, 'precipitation_prob': 1}, {'time': '2025-04-23 15:00:00', 'temp': 10.74, 'wind_speed': 5.4, 'humidity': 82, 'weather_desc': 'broken clouds', 'cloudiness': 78, 'precipitation_prob': 0}, {'time': '2025-04-23 18:00:00', 'temp': 11.45, 'wind_speed': 3.22, 'humidity': 79, 'weather_desc': 'light rain', 'cloudiness': 82, 'precipitation_prob': 0.2}, {'time': '2025-04-23 21:00:00', 'temp': 10.33, 'wind_speed': 1.82, 'humidity': 81, 'weather_desc': 'overcast clouds', 'cloudiness': 100, 'precipitation_prob': 0}, {'time': '2025-04-24 00:00:00', 'temp': 8.57, 'wind_speed': 1.61, 'humidity': 85, 'weather_desc': 'overcast clouds', 'cloudiness': 93, 'precipitation_prob': 0}, {'time': '2025-04-24 03:00:00', 'temp': 8.27, 'wind_speed': 2.1, 'humidity': 80, 'weather_desc': 'overcast clouds', 'cloudiness': 99, 'precipitation_prob': 0}, {'time': '2025-04-24 06:00:00', 'temp': 7

We also want a function for the current weather rather than the forecast. That way, we can get the entire day's weather reported in the morning brief.

In [9]:
load_dotenv() # keep this deprecated method just in dev

# Version 1: getting current weather of city
# endpoint: http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units={units}

units = 'metric'
city = 'London'
url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY_OPEN_WEATHER}&units={units}"
response = requests.get(url)
data = response.json()
current_weather = {}

current_weather['time'] = datetime.fromtimestamp(data['dt']).strftime('%Y-%m-%d %H:%M:%S')

print(current_weather['time'])
current_weather['temp'] = data['main']['temp']
current_weather['wind_speed'] = data['wind']['speed']
current_weather['humidity'] = data['main']['humidity']
current_weather['weather_desc'] = data['weather'][0]['description']
current_weather['cloudiness'] = data['clouds']['all']
current_weather['sunset_time'] = datetime.fromtimestamp(data['sys']['sunset']).strftime('%Y-%m-%d %H:%M:%S')

print(current_weather)

2025-04-23 12:34:40
{'time': '2025-04-23 12:34:40', 'temp': 10.2, 'wind_speed': 7.2, 'humidity': 85, 'weather_desc': 'broken clouds', 'cloudiness': 75, 'sunset_time': '2025-04-23 20:09:36'}


In [ ]:
# Function version of the above testing code
def get_current_weather_city(city: str):
    """ Get current weather for a city. Returns a dictionary of time, temperature, wind speed, humidity, weather description, cloudiness percentage and sunset time. 
    Args:
        city (str): city for weather forecast
    """
    # Note future extension to add in error handling for incorrect city names or error status code from API
    units = 'metric'
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY_OPEN_WEATHER}&units={units}"
    response = requests.get(url)
    data = response.json()
    current_weather = {}
    current_weather['time'] = datetime.fromtimestamp(data['dt']).strftime('%Y-%m-%d %H:%M:%S') # convert from unix
    current_weather['temp'] = data['main']['temp']
    current_weather['wind_speed'] = data['wind']['speed']
    current_weather['humidity'] = data['main']['humidity']
    current_weather['weather_desc'] = data['weather'][0]['description']
    current_weather['cloudiness'] = data['clouds']['all']
    current_weather['sunset_time'] = datetime.fromtimestamp(data['sys']['sunset']).strftime('%Y-%m-%d %H:%M:%S') #convert from unix
    return current_weather

print(get_current_weather_city('London'))


{'time': '2025-04-23 12:44:53', 'temp': 10.36, 'wind_speed': 7.2, 'humidity': 82, 'weather_desc': 'broken clouds', 'cloudiness': 75, 'sunset_time': '2025-04-23 20:09:36'}


In [ ]:
load_dotenv()

# Version 2: getting current weather of zipcode and country
# endpoint: https://api.openweathermap.org/data/2.5/weather?zip={zip code},{country code}&appid={API key}

def get_current_weather_zipcode(zip_code: str, country_code: str):
    """ Get current weather for a zip code with country code. Returns a dictionary of time, temperature, wind speed, humidity, weather description, cloudiness percentage and sunset time. 
    Args:
        zip_code (str): location zipcode (first part of uk zip, eg. e1 or nw1)
        country_code (str): country code of zipcode
    """
    # Note future extension to add in error handling for incorrect city names or error status code from API
    units = 'metric'
    url = f"https://api.openweathermap.org/data/2.5/weather?zip={zip_code},{country_code}&appid={API_KEY_OPEN_WEATHER}&units={units}"
    response = requests.get(url)
    data = response.json()
    current_weather = {}
    current_weather['time'] = datetime.fromtimestamp(data['dt']).strftime('%Y-%m-%d %H:%M:%S') # convert from unix
    current_weather['temp'] = data['main']['temp']
    current_weather['wind_speed'] = data['wind']['speed']
    current_weather['humidity'] = data['main']['humidity']
    current_weather['weather_desc'] = data['weather'][0]['description']
    current_weather['cloudiness'] = data['clouds']['all']
    current_weather['sunset_time'] = datetime.fromtimestamp(data['sys']['sunset']).strftime('%Y-%m-%d %H:%M:%S') #convert from unix
    return current_weather

print(get_current_weather_zipcode("wc1e", 'gb'))

{'time': '2025-04-23 12:55:22', 'temp': 10.53, 'wind_speed': 7.2, 'humidity': 82, 'weather_desc': 'broken clouds', 'cloudiness': 75, 'sunset_time': '2025-04-23 20:09:36'}


We liely still want historical weather because we want the morning brief to say how much the temperature changed from yesterday. To do this, we should probably use the historical open weather API.

In [ ]:
load_dotenv() # keep this deprecated method just in dev

